# Phase 3 Weighted Bagging

In [53]:
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import os
import re
import csv
import codecs
import gensim
import itertools
import numpy as np
import pandas as pd
import operator
import sys

from nltk import ngrams
from collections import Counter
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from iwillwin.data_utils.feature_engineering import FeatureCreator

from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize

import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
from os import listdir
from os.path import isfile, join

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from keras.utils import to_categorical

In [54]:
NB_WORDS, MAX_SEQUENCE_LENGTH = 50000, 50
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains_nns, tests_nns, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [55]:
trains_meta = trains_nns[2]
tests_meta = tests_nns[2]

In [56]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [57]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [58]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains_nns[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests_nns[2], test_has_rumor.reshape((-1, 1))), axis=1)

In [59]:
oof_file_names = sorted([f for f in listdir('../data/p_ensemble/oofs/') if isfile(join('../data/p_ensemble/oofs/', f)) and f != '.gitkeep'])
preds_file_names = [name.replace('-Train', '') for name in oof_file_names]

oofs = []
preds = []
for name in oof_file_names:
    oofs.append(pd.read_csv('../data/p_ensemble/oofs/' + name))
for name in preds_file_names:
    preds.append(pd.read_csv('../data/p_ensemble/preds/' + name))    

In [60]:
for i, name in enumerate(oof_file_names):
    print(i, name)
    
trains = pd.DataFrame()
tests = pd.DataFrame()

for i in range(len(oof_file_names)):
    for label_type in ['agreed', 'disagreed', 'unrelated']:
        trains['oofs_{}_{}'.format(i, label_type)] = oofs[i][label_type].values
        tests['oofs_pred{}_{}'.format(i, label_type)] = preds[i][label_type].values

0 AddNN-Ensemble-Weighted-StopOnAcc-Train-L0.875371-NB50000.csv
1 AddNN-Ensemble-Weighted-StopOnLoss-Train-L0.867090-NB50000.csv
2 LightGBM-Ensemble-Train-L0.866327-NB50000.csv
3 LightGBMWordLevel-Ensemble-Train-L0.861769-NB50000.csv
4 Logistic-Ensemble-Train-L0.866449-NB50000.csv
5 Ridge-Ensemble-Train-L0.866237-NB50000.csv


In [61]:
unrelated = pd.DataFrame()
agreeds = pd.DataFrame()
disagreeds = pd.DataFrame()

#check_oofs = True
check_oofs = False


if check_oofs:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = oofs[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = oofs[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = oofs[i]['disagreed'].values
else:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = preds[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = preds[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = preds[i]['disagreed'].values  

In [62]:
agreeds.corr()

,oofs_agreed_0,oofs_agreed_1,oofs_agreed_2,oofs_agreed_3,oofs_agreed_4,oofs_agreed_5
oofs_agreed_0,1.000000,0.999986,0.999710,0.997443,0.994811,0.998455
oofs_agreed_1,0.999986,1.000000,0.999740,0.997455,0.994560,0.998365
oofs_agreed_2,0.999710,0.999740,1.000000,0.997434,0.993696,0.998148
oofs_agreed_3,0.997443,0.997455,0.997434,1.000000,0.989557,0.995106
oofs_agreed_4,0.994811,0.994560,0.993696,0.989557,1.000000,0.995949
oofs_agreed_5,0.998455,0.998365,0.998148,0.995106,0.995949,1.000000


In [63]:
disagreeds.corr()

,oofs_disagreeds_0,oofs_disagreeds_1,oofs_disagreeds_2,oofs_disagreeds_3,oofs_disagreeds_4,oofs_disagreeds_5
oofs_disagreeds_0,1.000000,0.976072,0.980943,0.977130,0.911610,0.920323
oofs_disagreeds_1,0.976072,1.000000,0.997218,0.994303,0.974962,0.962633
oofs_disagreeds_2,0.980943,0.997218,1.000000,0.995663,0.968548,0.956897
oofs_disagreeds_3,0.977130,0.994303,0.995663,1.000000,0.964323,0.953642
oofs_disagreeds_4,0.911610,0.974962,0.968548,0.964323,1.000000,0.950559
oofs_disagreeds_5,0.920323,0.962633,0.956897,0.953642,0.950559,1.000000


In [64]:
unrelated.corr()

,oofs_unrelated_0,oofs_unrelated_1,oofs_unrelated_2,oofs_unrelated_3,oofs_unrelated_4,oofs_unrelated_5
oofs_unrelated_0,1.000000,0.992047,0.992922,0.989629,0.980826,0.992267
oofs_unrelated_1,0.992047,1.000000,0.999537,0.997011,0.992254,0.999502
oofs_unrelated_2,0.992922,0.999537,1.000000,0.996987,0.990938,0.999046
oofs_unrelated_3,0.989629,0.997011,0.996987,1.000000,0.986483,0.996372
oofs_unrelated_4,0.980826,0.992254,0.990938,0.986483,1.000000,0.991760
oofs_unrelated_5,0.992267,0.999502,0.999046,0.996372,0.991760,1.000000


# Prepare Different Inputs

In [65]:
# Only use oofs
ensemble_trains = trains.values
ensemble_tests = tests.values

# Ensemble With NN

In [66]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

In [67]:
from keras import regularizers

def weighted_accuracy(y_true, y_pred):
    # hard version, 0.5 counts as correct
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    label_weights = K.argmax(weight_mask, axis=-1)
    
    true_label = K.argmax(y_true, axis=-1)
    pred_label = K.argmax(y_pred, axis=-1)
    
    res = K.cast(K.equal(true_label, pred_label), tf.int64) * label_weights / K.sum(label_weights)
    res = K.sum(res)
    return res

def get_dense_add_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.1)(new_features)
        features = Concatenate()([features, new_features])

    h = Highway(activation='relu')(features)
    out_ = Dense(3, activation='softmax')(h)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [68]:
def get_logit_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")    
    out_ = Dense(3, activation='softmax')(features_inputs)
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [69]:
def _agent_get_model():
    return get_dense_add_net(ensemble_trains.shape[1])

In [70]:
def np_weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    weight_mask = np.max(weight_mask, axis=-1)
    norms = np.sum(weight_mask)
    
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(y_pred, axis=-1)
    
    res = ((y_true == y_pred) * weight_mask).sum() / norms
    return res

In [71]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32') # Hard
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

In [72]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    label_weights = K.max(K.cast(weight_mask, 'float32'), axis=-1)
    
    true_label = K.argmax(y_true, axis=-1)
    pred_label = K.argmax(y_pred, axis=-1)
    
    res = K.cast(K.equal(true_label, pred_label), tf.float32) * label_weights / K.sum(label_weights)
    res = K.sum(res)
    return res

In [73]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, features, y, fold_count, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        weight_val=scale_sample_weight
        class_weight=None
        fold_size = len(features) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(features)

            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])
            
            val_features = features[fold_start:fold_end]
            val_y = y[fold_start:fold_end]
            fold_pos = (np.sum(train_y) / len(train_features))

            train_data = {
                "mata-features": train_features,
            }

            val_data = {
                "mata-features": val_features,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         verbose=2,
                         class_weight={0: 1/16, 1: 1/15, 2: 1/5},
                         callbacks=[early_stopping, model_checkpoint],)
        bst_val_score = max(hist.history['val_weighted_accuracy'])
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [74]:
trainer = KerasModelTrainer(model_stamp="Ensemble-DenseNet", epoch_num=500)
models, score, folds_preds = trainer.train_folds(features=ensemble_trains, y=to_categorical(labels), augments=None, fold_count=10,
    batch_size=1024, 
    scale_sample_weight=None, class_weight=None,
    get_model_func=_agent_get_model, 
    patience=30)

C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 18)           0                                            
__________________________________________________________________________________________________
dense_145 (Dense)               (None, 24)           456         mata-features[0][0]              
__________________________________________________________________________________________________
dropout_121 (Dropout)           (None, 24)           0           dense_145[0][0]                  
__________________________________________________________________________________________________
concatenate_121 (Concatenate)   (None, 42)           0           mata-features[0][0]              
                                                                 dropout_121[0][0]                
__________

Epoch 30/500
 - 1s - loss: 0.0192 - acc: 0.8831 - weighted_accuracy: 0.8750 - val_loss: 0.2376 - val_acc: 0.8957 - val_weighted_accuracy: 0.8857
Epoch 31/500
 - 1s - loss: 0.0192 - acc: 0.8833 - weighted_accuracy: 0.8750 - val_loss: 0.2337 - val_acc: 0.8963 - val_weighted_accuracy: 0.8858
Epoch 32/500
 - 1s - loss: 0.0192 - acc: 0.8833 - weighted_accuracy: 0.8750 - val_loss: 0.2393 - val_acc: 0.8938 - val_weighted_accuracy: 0.8853
Epoch 33/500
 - 1s - loss: 0.0192 - acc: 0.8836 - weighted_accuracy: 0.8753 - val_loss: 0.2338 - val_acc: 0.8964 - val_weighted_accuracy: 0.8856
Epoch 34/500
 - 1s - loss: 0.0192 - acc: 0.8833 - weighted_accuracy: 0.8751 - val_loss: 0.2364 - val_acc: 0.8952 - val_weighted_accuracy: 0.8853
Epoch 35/500
 - 1s - loss: 0.0192 - acc: 0.8833 - weighted_accuracy: 0.8750 - val_loss: 0.2353 - val_acc: 0.8952 - val_weighted_accuracy: 0.8854
Epoch 36/500
 - 1s - loss: 0.0192 - acc: 0.8835 - weighted_accuracy: 0.8751 - val_loss: 0.2344 - val_acc: 0.8957 - val_weighted_ac

Epoch 18/500
 - 2s - loss: 0.0189 - acc: 0.8855 - weighted_accuracy: 0.8768 - val_loss: 0.2780 - val_acc: 0.8759 - val_weighted_accuracy: 0.8704
Epoch 19/500
 - 2s - loss: 0.0189 - acc: 0.8854 - weighted_accuracy: 0.8768 - val_loss: 0.2885 - val_acc: 0.8703 - val_weighted_accuracy: 0.8682
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 18)           0                                            
__________________________________________________________________________________________________
dense_169 (Dense)               (None, 24)           456         mata-features[0][0]              
__________________________________________________________________________________________________
dropout_141 (Dropout)           (None, 24)           0           dense_169[0][0]                  
_________________

Epoch 17/500
 - 2s - loss: 0.0191 - acc: 0.8850 - weighted_accuracy: 0.8762 - val_loss: 0.2701 - val_acc: 0.8805 - val_weighted_accuracy: 0.8754
Epoch 18/500
 - 2s - loss: 0.0190 - acc: 0.8849 - weighted_accuracy: 0.8760 - val_loss: 0.2716 - val_acc: 0.8802 - val_weighted_accuracy: 0.8752
Epoch 19/500
 - 2s - loss: 0.0190 - acc: 0.8847 - weighted_accuracy: 0.8760 - val_loss: 0.2690 - val_acc: 0.8810 - val_weighted_accuracy: 0.8758
Epoch 20/500
 - 2s - loss: 0.0190 - acc: 0.8847 - weighted_accuracy: 0.8761 - val_loss: 0.2712 - val_acc: 0.8792 - val_weighted_accuracy: 0.8753
Epoch 21/500
 - 2s - loss: 0.0190 - acc: 0.8848 - weighted_accuracy: 0.8763 - val_loss: 0.2661 - val_acc: 0.8817 - val_weighted_accuracy: 0.8755
Epoch 22/500
 - 2s - loss: 0.0190 - acc: 0.8849 - weighted_accuracy: 0.8764 - val_loss: 0.2662 - val_acc: 0.8820 - val_weighted_accuracy: 0.8747
Epoch 23/500
 - 2s - loss: 0.0190 - acc: 0.8847 - weighted_accuracy: 0.8760 - val_loss: 0.2672 - val_acc: 0.8809 - val_weighted_ac

                                                                 dropout_161[0][0]                
__________________________________________________________________________________________________
dense_194 (Dense)               (None, 24)           1032        concatenate_161[0][0]            
__________________________________________________________________________________________________
dropout_162 (Dropout)           (None, 24)           0           dense_194[0][0]                  
__________________________________________________________________________________________________
concatenate_162 (Concatenate)   (None, 66)           0           concatenate_161[0][0]            
                                                                 dropout_162[0][0]                
__________________________________________________________________________________________________
dense_195 (Dense)               (None, 24)           1608        concatenate_162[0][0]            
__________

In [75]:
score

0.8771482783241311

In [76]:
print("score", score)
oofs_dir = "../data/p_ensemble/second_level/oofs/"
output_dir = "../data/p_ensemble/second_level/preds/"
onehot_pred_dir = "../data/p_ensemble/second_level/nn_one_hot/"

model_submit_prefix = "AddNN-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
train_predicts = np.concatenate(folds_preds, axis=0)
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict({"mata-features": ensemble_tests}, batch_size=128, verbose=1)
    test_predicts_list.append(test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8771482783241311
Predicting training results...
Predicting testing results...
80126/80126 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 10us/step
Predicting labeled testing results...


In [52]:
sub['Category'].value_counts() / len(sub)

unrelated    0.638569
agreed       0.334261
disagreed    0.027170
Name: Category, dtype: float64

In [ ]:
sub.to_csv("../data/high_ground/fine_tuned_nns.csv", index=False)